# Agentic Vector Store Interaction

This notebook contains experiments regarding making an agent interact with tools that execute operations on a vector store (QDrant)

In [2]:
# Importing necessary libraries
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph

from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
# Create dummy tools
@tool
def insert_candidate(name: str, email: str, resume: str):
    """Use this tool to insert a candidate into the vector database."""
    return f"Candidate {name} with email {email} and resume {resume} inserted successfully."

@tool
def search_candidate(query: str):
    """Use this tool to search for a candidate in the vector database according to the query."""
    return f"Candidate that satisfies the query '{query}' searched successfully."

@tool
def insert_company(name: str, company_name: str, job_title: str, company_query: str):
    """Use this tool to insert a company along with its query and wanted job title into the vector database."""
    return f"Company {company_name} that wants a {job_title} and with the query {company_query} inserted successfully."

@tool
def search_company(query: str):
    """Use this tool to search for a company in the vector database according to the query from the job seeker."""
    return f"Company that satisfies the query '{query}' searched successfully."

In [5]:
# Create a model
model = init_chat_model(
    "gpt-4o",
    model_provider="openai",
    temperature=0,
    verbose=True
)

tools = [insert_candidate, search_candidate, insert_company, search_company]
tool_node = ToolNode(tools)

# Bind tools to the LLM
model_with_tool = model.bind_tools(tools)

In [6]:
# Create a system prompt
system_prompt = """
You are a helpful talent hunter that helps job seekers find their dream jobs.
You are given a job seeker's query and you need to find the best job for them.
Additionally, you also help companies find the best candidates for their jobs.

You have access to the following tools:
- insert_candidate: to insert a candidate into the vector database
- search_candidate: to search for a candidate in the vector database
- insert_company: to insert a company into the vector database
- search_company: to search for a company in the vector database

You need to use the tools to find the best job for the job seeker.
You need to use the tools to find the best candidates for the company.

The tools may require some information to be inserted into the vector database.
You need to ask the job seeker for the information if the tools require it.

Sometimes, the job that a job seeker is looking for is not available in the vector database.
In this case, just inform the job seeker that the job is not available and ask them to come back later.

Likewise, if a suitable candidate is not available for a company, just inform the company that the candidate is not available and ask them to come back later.
"""

# Create a prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}")
])

# Create a model with tools
llm_chain = prompt_template | model_with_tool

In [7]:
# Create a function to call the model
def call_model(state: MessagesState, config: RunnableConfig):
    response = llm_chain.invoke({"input": state["messages"]})
    return {"messages": response}

# Create memory saver(memory)
memory = MemorySaver()

# Create a graph
agent_graph = StateGraph(state_schema=MessagesState)

# Add graph nodes
agent_graph.add_node("model", call_model)
agent_graph.add_node("tools", tool_node)

# Add edges
agent_graph.add_edge(START, "model")
agent_graph.add_conditional_edges("model", tools_condition, ["tools", END])
agent_graph.add_edge("tools", "model")

# Compile the graph
graph = agent_graph.compile(checkpointer=memory)

In [8]:
# Initiate a session
config = RunnableConfig(
        configurable={
            "thread_id": "thread_1",
            "user_id": "user_1",
            "temperature": 0,
        },
        recursion_limit=25
    )

In [9]:
human_message = HumanMessage(content="Hello! I am a job seeker looking for a job.")

result = graph.invoke({"messages": [human_message]}, config)

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Hi there! I'd be happy to help you find a job. Could you please provide me with some more details about the type of job you're looking for, your skills, and any other relevant information? This will help me search for the best opportunities for you.


In [10]:
message = """
My name is John Doe. I am a software engineer with 5 years of experience in the industry. I am looking for a job in the field of software engineering.
"""
human_message = HumanMessage(content=message)

result = graph.invoke({"messages": [human_message]}, config)

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
I have successfully searched for companies looking for a software engineer with 5 years of experience. However, it seems that there are no current job openings matching your query in the database at the moment.

I recommend checking back later or providing more specific details about your skills or preferences, which might help in finding a more suitable match. If you have a resume or any additional information you'd like to share, I can also insert your profile into the database for future opportunities. Let me know how you'd like to proceed!
